In [1]:
import time
import pyspark
import numpy as np

In [2]:
file_name = 'household_power_consumption.txt'
sc = pyspark.SparkContext(appName='sample_code_1')
text_file = sc.textFile(file_name)

In [3]:
take_size = 3
attr_idx = [2, 3]
bc_attr_idx = sc.broadcast(attr_idx)
bc_attr_idx.value

[2, 3]

In [4]:
# show data
text_file.take(take_size)

['Date;Time;Global_active_power;Global_reactive_power;Voltage;Global_intensity;Sub_metering_1;Sub_metering_2;Sub_metering_3',
 '16/12/2006;17:24:00;4.216;0.418;234.840;18.400;0.000;1.000;17.000',
 '16/12/2006;17:25:00;5.360;0.436;233.630;23.000;0.000;1.000;16.000']

In [5]:
def get_attrs(row):
    attrs = row.split(';')
    return np.take(attrs, bc_attr_idx.value)

def trans_type(row):
    try:
        return [np.double(value) for value in row]
    except:
        return [np.nan] * len(row)

text_file.map(get_attrs).map(trans_type).take(take_size)

[[nan, nan], [4.216, 0.418], [5.36, 0.436]]

In [6]:
def get_max(row1, row2):
    stacked = np.vstack([row1, row2])
    return np.nanmax(stacked, axis=0)

text_file.map(get_attrs).map(trans_type).reduce(get_max)

array([11.122,  1.39 ])

In [7]:
# dont believe this result 

start = time.time()
result = text_file.map(get_attrs).map(trans_type).reduce(get_max)
print('result: {}'.format(result))
print('time: {}'.format(time.time() - start))

result: [11.122  1.39 ]
time: 42.419591665267944
